# Proyecto

# Libraries

In [8]:
import serial
import sys
import glob
import time
import numpy as np

# Functions

In [20]:
def listSerialPorts():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

def connectToPort(n):
    ports = listSerialPorts()
    return serial.Serial(ports[n])

def readValueFromData(data):
    # +000.000/0 G S
    dataString = data.decode("utf-8").replace('\r\n','')
    if dataString.endswith(' G S'):
        result = dataString.replace(' G S','').replace('/','').replace('+','')
        return float(result)
    else:
        raise Exception("Invalied format")
    
def serialRead(connection):
    while True:
          try:
              connection.flushInput()
              return readValueFromData(connection.readline())
          except:
              pass
    

# Connection

In [54]:

ports = listSerialPorts()

for i in range(0,len(ports)):
    print('{0}: {1}'.format(i+1, ports[i]))

1: /dev/tty.wlan-debug
2: /dev/tty.BT-163
3: /dev/tty.Bluetooth-Incoming-Port


In [ ]:
port = 3

connection = connectToPort(port)

In [17]:
# for i in range(0,10):
#     serialRead(connection)
#     time.sleep(1)

2.0
0.0001
200.0001
200.0001
200.0
200.0
200.0
200.0
200.0
200.0


In [18]:
# connection.close()

# Procedure

Data

In [2]:
scaleDivision = 0.0001 # g
minimumCharge = 0.01 # g
nominalCalibrationMass = 200 # g
EMPs = { # EMP para masa E1 según la OIML R 111
    10: 0.00002,
    20: 0.000025,
    40: 0.00005,
    60: 0.00005,
    80: 0.000075,
    100: 0.00005,
    120: 0.000075,
    140: 0.0001,
    160: 0.0001,
    180: 0.00125,
    200: 0.0001,
    220: 0.000125
}


In [4]:
import random

In [9]:
print('Iniciando procedimiento de calibración de la balanza.')
input('Presione enter para continuar.')

# PASO 1

print('''
      Paso 1: Prueba de excentricidad
      
      Se debe tomar un patrón de peso de 200 g y utilizarse en todos los puntos de la prueba conforme se indique en pantalla.
      El patrón no se debe cambiar bajo ninguna circunstancia.
      ''')

input('Presione enter para continuar.')

points = ['Central','Inferior izquierdo','Superior izquierdo','Superior derecho','Inferior derecho','Central']

eccentricityPoints = []
for i in range(0,len(points)):
    print('\nCalibración del punto {0}, favor posicionar el patrón en el punto {1}'.format(i+1,points[i]))
    keep = True
    currentPoint = []
    while keep:
        value = 200+random.uniform(-2, 2)# serialRead(connection)
        if value >= nominalCalibrationMass*0.9:
            keep = False
            currentPoint.append(value)
    print('\nFavor retire el partón de la balanza.')
    keep = True
    while keep:
        value = 0+random.uniform(-1, 1)# serialRead(connection)
        if value <= nominalCalibrationMass*0.1:
            keep = False
            currentPoint.append(value)
    
    eccentricityPoints.append(currentPoint)
    
eccentricityPoints = np.array(eccentricityPoints)

p0 = eccentricityPoints[-1,0]

# print(eccentricityPoints)

# print((np.vectorize(lambda row: np.abs(row[0]-row[1]-p0)))(eccentricityPoints))

# print(np.apply_along_axis(lambda row: np.abs(row[0]-row[1]-p0), 1, eccentricityPoints))

dEccentricityMax = np.max(np.apply_along_axis(lambda row: np.abs(row[0]-row[1]-p0), 1, eccentricityPoints))

print(dEccentricityMax)

# PASO 2

       

Iniciando procedimiento de calibración de la balanza.

      Paso 1: Prueba de excentricidad
      
      Se debe tomar un patrón de peso de 200 g y utilizarse en todos los puntos de la prueba conforme se indique en pantalla.
      El patrón no se debe cambiar bajo ninguna circunstancia.
      

Calibración del punto 1, favor posicionar el patrón en el punto Central

Favor retire el partón de la balanza.

Calibración del punto 2, favor posicionar el patrón en el punto Inferior izquierdo

Favor retire el partón de la balanza.

Calibración del punto 3, favor posicionar el patrón en el punto Superior izquierdo

Favor retire el partón de la balanza.

Calibración del punto 4, favor posicionar el patrón en el punto Superior derecho

Favor retire el partón de la balanza.

Calibración del punto 5, favor posicionar el patrón en el punto Inferior derecho

Favor retire el partón de la balanza.

Calibración del punto 6, favor posicionar el patrón en el punto Central

Favor retire el partón de la b

In [17]:
nominalCalibrationMass

200

In [70]:
# Datos de la prueba de repetibilidad

repetic = np.arange(1,11,1)

print(f"Iniciando prueba de repetibilidad para {repetic.size} mediciones")
input("Presione enter para continuar")

print('''Paso 1: Carga y descarga al 50%
      
      info
      
      ''')

input("Presione enter para continuar")


carga_50 = nominalCalibrationMass/2 # carga al 50% 


# CARGA AL 50 %
repetib50 = []

for i in repetic:
    
      print(f"Coloque la carga {i}:")
      keep = True
      ind50 = []
      while keep:   # carga al 50
            value = carga_50 + np.random.uniform(-2,2) # serialRead(connection)
            if value >= carga_50*0.9:
                  keep = False
                  ind50.append(value)
      print(f"\nFavor retirar la carga")

      keep = True
      while keep:
            value = 0 + random.uniform(-1,1)
            if value <= carga_50*0.1:
                  keep = False
                  ind50.append(value)
      
      repetib50.append(ind50)

repetib50 = np.array(repetib50)



# CARGA AL 100%
repetib100 = []

print("Paso 2: carga y descarga al 100%")
input("Presione enter para continuar")


for i in repetic:
    
      print(f"Coloque la carga {i}:")
      keep = True
      ind100 = []
      while keep:   
            value = nominalCalibrationMass + np.random.uniform(-2,2) # serialRead(connection)
            if value >= nominalCalibrationMass*0.9:
                  keep = False
                  ind100.append(value)
      print(f"\nFavor retirar la carga")

      keep = True
      while keep:
            value = 0 + random.uniform(-1,1)
            if value <= nominalCalibrationMass*0.1:
                  keep = False
                  ind100.append(value)
      
      repetib100.append(ind100)

repetib100 = np.array(repetib100)


# Desviaciones estandar

S_Carg50 = np.std(repetib50[:,0], ddof=1) # desviacion estandar muestral de la carga a 50%
S_desc50 = np.std(repetib50[:,1], ddof=1) # desviacion estandar muestral de la descarga a 50%
S_Carg100 = np.std(repetib100[:,0], ddof=1) # desviacion estandar muestral de la carga a 100%
S_desc100 = np.std(repetib100[:,1], ddof=1) # desviacion estandar muestral de la descarga a 100%

# Desviaciones en array
desv = np.zeros((1,4))
desv = np.append(desv, [[S_Carg50, S_desc50, S_Carg100, S_desc100]], axis=0)[1]

Iniciando prueba de repetibilidad para 10 mediciones
Paso 1: Carga y descarga al 50%
      
      info
      
      
Coloque la carga 1:

Favor retirar la carga
Coloque la carga 2:

Favor retirar la carga
Coloque la carga 3:

Favor retirar la carga
Coloque la carga 4:

Favor retirar la carga
Coloque la carga 5:

Favor retirar la carga
Coloque la carga 6:

Favor retirar la carga
Coloque la carga 7:

Favor retirar la carga
Coloque la carga 8:

Favor retirar la carga
Coloque la carga 9:

Favor retirar la carga
Coloque la carga 10:

Favor retirar la carga
Paso 2: carga y descarga al 100%
Coloque la carga 1:

Favor retirar la carga
Coloque la carga 2:

Favor retirar la carga
Coloque la carga 3:

Favor retirar la carga
Coloque la carga 4:

Favor retirar la carga
Coloque la carga 5:

Favor retirar la carga
Coloque la carga 6:

Favor retirar la carga
Coloque la carga 7:

Favor retirar la carga
Coloque la carga 8:

Favor retirar la carga
Coloque la carga 9:

Favor retirar la carga
Coloque la ca

In [71]:
# Info extra

import pandas as pd

rep50 = pd.DataFrame(repetib50, columns= ["Carga al 50%", "Descarga al 50%"])
rep100 = pd.DataFrame(repetib100 , columns= ["Carga al 100%", "Descarga al 100%"])

Datos_rep = pd.concat([rep50,rep100], axis= 1)

Datos_rep

,Carga al 50%,Descarga al 50%,Carga al 100%,Descarga al 100%
0,100.055693,-0.929857,198.250776,0.964207
1,98.078411,0.440052,200.405385,-0.307043
2,101.635595,0.109874,199.247917,-0.261179
3,99.216989,-0.821779,199.218991,-0.986528
4,101.901387,-0.515923,198.050555,-0.847875
5,100.753514,0.316598,198.656168,0.603595
6,100.321375,-0.646777,199.370588,-0.182947
7,98.787684,0.892916,200.776624,0.318559
8,98.355863,0.347013,198.264264,0.354075
9,98.217217,0.708339,198.713059,0.488787
